In [ ]:
!pip install kora -q
from kora.selenium import wd
 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import json
import re

from bs4 import BeautifulSoup
import requests

import pickle

#path for saved dataset
path = '/content/gdrive/My Drive/4sight/'
dataset_filename = "Lista-Dataset.pickle"

domain = "https://lista.mt"
root = "https://lista.mt"

company_index = 0
employee_index = 0

company_list = []
employee_list = []

src_set = set()

class Company:
  def __init__(self, id, name, industry, email, phone, website, fax, employees, address, src):
    self.id = id
    self.name = name
    self.industry = industry
    self.email = email
    self.phone = phone
    self.website = website
    self.fax = fax
    self.address = address
    self.src = src
    if employees is None:
      employees = []
    self.employees = employees
  def add_employee(self, employee):
    self.employees.append(employee)
  def get_employees_ids(self):
    ids = []
    for employee in self.employees:
      ids.append(employee.id)
    return ids

def generate_company_id():
  global company_index
  new_id = 'Lista-CO.' + '{:0>4}'.format(company_index)
  company_index += 1
  return new_id
  
class Employee:
  def __init__(self, id, name, position, email, mobile, phone, company):
    self.id = id
    self.name = name
    self.position = position
    self.email = email
    self.mobile = mobile
    self.phone = phone
    self.company = company
    company.add_employee(self)

def generate_employee_id():
  global employee_index
  new_id = 'MP-EE.' + '{:0>4}'.format(employee_index)
  employee_index += 1
  return new_id

class Address:
  def __init__(self, street1, street2, zip, city, country):
    self.street1 = street1
    self.street2 = street2
    self.zip = zip
    self.city = city
    self.country = country

html = requests.get(root).text
soup = BeautifulSoup(html, 'lxml')

def get_urls(domain,max_page):
  urls = []
  for num in range(max_page):
    param = "/?pagenum=" + str(num + 1) + "&sort%5B1%5D=asc"
    link = domain + param
    print(link)
    html = requests.get(link).text
    soup = BeautifulSoup(html,'lxml')
    tds = soup.find_all('td',{'data-label':'More Info'})
    for td in tds:
      try:
        urls.append(td.find('a')['href'])
      except:
        print('no url')

  return urls

def add_company(urls):
  for url in urls:
    try:
      if url not in src_set:
        src_set.add(url)
        wd.get(url)
        wd.close()
        html = wd.page_source
        soup = BeautifulSoup(html ,'lxml')
        #name
        try:
          name = soup.find('tr',{'class':'gv-field-1-1'}).find('td').text
          #print(name)
        except:
          name = ''
        #phone
        try:
          phone = soup.find('tr',{'class':'gv-field-1-4'}).find('a')['href'][4:].replace('%20','')
          phone = phone.split('/')[0].split(',')[0]
          #print(phone)
        except:
          phone =''
        #website 
        try:
          website =soup.find('tr',{'class':'gv-field-1-5'}).find('a')['href']
          #print(website)
        except:
          website=''
        #email
        try:
          email = soup.find('tr',{'class':'gv-field-1-6'}).find('a')['href']
          #print(email)
        except:
          email = ''
        #address
        try:
          address_str = soup.find('tr',{'class':'gv-field-1-15'}).find('td').text[:-6]
          street1 = address_str
          address = Address(street1,None,None,None,'Malta')
          #print(address)
        except:
          address = Address(None,None,None,None,'Malta')
        #industry
        try:
          industries = soup.find('tr',{'class':'gv-field-1-20'}).find_all('li')
          industry =''
          for i in industries:
            industry += (i.text + ' ')
          #print(industry)
        except:
          industry =''
        company = Company(generate_company_id(),name,industry,email,phone,website,None,None,address,url)
        company_list.append(company)
        print(company.name , 'added')
    except Exception as e:
      print(e)

urls = get_urls(domain,1)
add_company(urls)
wd.quit()
print(src_set)
print('count', len(src_set))
dataset = [company_list,employee_list]

#save dataset
with open(path + dataset_filename, 'wb') as f:
  pickle.dump(dataset,f,pickle.HIGHEST_PROTOCOL)

#for c in dataset[0]:
  #print("Company: ", c.id,c.name,c.industry,c.get_employees_ids())
  #print('         ',c.address.street1,c.address.city,c.email,c.phone,c.fax,c.website)
#for e in dataset[1]:
  #print("Employee: ",e.id, e.name,e.position,e.company.id,e.company.name, e.email)

Mounted at /content/gdrive
https://lista.mt/?pagenum=1&sort%5B1%5D=asc
HTTPConnectionPool(host='127.0.0.1', port=46595): Max retries exceeded with url: /session/f92f6cb4a8693d5eae2786d66873eac7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9d2cbebd30>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPConnectionPool(host='127.0.0.1', port=46595): Max retries exceeded with url: /session/f92f6cb4a8693d5eae2786d66873eac7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9d2cbebe80>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPConnectionPool(host='127.0.0.1', port=46595): Max retries exceeded with url: /session/f92f6cb4a8693d5eae2786d66873eac7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9d2cbeb278>: Failed to establish a new connection: [Errno 111] Connection refused',))
HTTPConnectionPool(host='127.0.0.1', port=46595): 